# Lab automation with Python (and 3d printers): Do more for less<a class="tocSkip">
---
    
## I. Interfacing: `.dll` is all you need<a class="tocSkip">
## II. Actuators: cheap is (sometimes) enough<a class="tocSkip">
## III. Alignment is boring, let the computer work<a class="tocSkip">

## Disclaimer<a class="tocSkip">

    
**The goal is to let you know what solutions exist, not to go into details**
    
**There are many tutorials to show you how to replace Matlab/Labview with Python for free, I just show here some tricks I found useful.**

# Using `.dll` with Ctypes

If there is no Python/Matalb/Labview specific module for a given hardware, you can still use a `.dll` (Windows) / `.so` (Linux) with Python 

## OK, but what is a `.dll`?

A `.dll` is a compiled file, but unlike a `.exe`, it does nothing (no main function).

It is a librairy of functions that can be called by an external program. 

In a typical user interface to control a device, a graphical interface call functions from a `.dll` to communicate with the device.

## ctypes

`ctypes` module allows:
1. Calling function of a library within Python
2. Interfacing with C variables

**Example: [ALP4lib](https://github.com/wavefrontshaping/ALP4lib) to control [Vialux](https://www.vialux.de/en/) DMD modulators**

### Load a `.dll`

In [1]:
import ctypes as ct

On windows

```python
libPath = './mylib.dll'
lib = ct.CDLL(libPath)
```

### Use functions

Use functions *almost* like normal Python function
```python
lib.function()
```

### Data types (the *almost* part)

Look at the API documentation and/or the header file (`.h` file)

**Example:**
```C
long AlpDevHalt(unsigned long device_id);
```

This function takes a integer of type `unsigned long` as input, and return a `long` integer

```python
device_id = 10 # Python variable
ret = lib.AlpDevHalt(ct.ulong(device_id))
print(ret.value())
```

most of the time, it should work too:

```python
ret = lib.AlpDevHalt(device_id)
```

### Difficult part, pointers and arrays


We sometimes have to send/receive array of values (e.g. for modulators or cameras).

A DMD function to upload a mask (that will later be displaied on the modulator):

```C
long AlpSeqPut(unsigned long DeviceId, long PicLoad, void *UserArrayPtr);
```

This function wants a pointer, but to what?

**Read the docs**

In the docs:

![](./images/resolution.png)


![](./images/size.png)

Type is `char unsigned` (i.e. unsigned int on 8 bits, or uint8)

**Cast Python/numpy arrays to C array**

Numpy does that!

I want to send one mask (`PicLoad` = 1)
```python
rows = 768
columns = 1024

mask = np.randint(2**8, size = (raws,columns))
C_mask = mask.astype(np.uint8).ctypes.data_as(ct.c_void_p)
```

**Pass the array to the function**

```python
lib.AlpSeqPut(device_id, ct.long(1), C_mask)
```

## That's it!<a class="tocSkip">
    
### You can now control any instrument in Python as long as you have a `.dll`!<a class="tocSkip">

### And because it is that *simple*, 90% of the time, somebody already did it. Check online first!<a class="tocSkip">

# DIY: Cheap automation components

## **Problem:** Not everything requires precise/fast instruments<a class="tocSkip">

## Example 1: Blocking/unblocking a reference arm

### Commercial optical shutter

![](./images/shutter_TL_full.png)

[//]: # (<img src="./images/shutter_TL_specs.gif" alt="drawing" style="width:500px;"/>)

### DIY

![](./images/shutter_DIY.png)

* 3d files: [github.com/wavefrontshaping/Servo-Motor-Shutter](https://github.com/wavefrontshaping/Servo-Motor-Shutter)
* Software (PC/microcontroller): [github.com/wavefrontshaping/Optical-Hardware-Control](https://github.com/wavefrontshaping/Optical-Hardware-Control)

### **Microcontroller with Python! (i.e. Adafruit cards, Raspberry Pi Pico, ...)**<a class="tocSkip">

![](./images/animation_shutter.gif)

<img src="./images/shutter_setup.jpg" alt="drawing" style="width:500px;"/>

## Example 2: Control laser power rotating a polarizer

Rotation of optical elements may require precision, but not too much if it is only to control the input power (to desaturate a camera for instance).

### Commercial optical shutter


![](./images/rotator_TL.png)

<img src="./images/rotator_TL_specs.png" alt="drawing" style="width:400px;"/>



### DIY

<img src="./images/rotator_DIY.png" alt="drawing" style="width:900px;"/>

<img src="./images/rotator_DIY_1.png" alt="drawing" style="width:750px;"/>

**<center>Precision: ~1-2 degrees</center>**

# Alignment is boring, let the computer work

## Precision is sometimes crucial

### An example: Measuring the transmission matrix of a multimode fiber<a class="tocSkip">

## The hard(ware) way

<img src="./images/MMF/1.png" alt="drawing" style="width:1500px;"/>

### Setups can be complicated and difficult to align

<img src="./images/MMF/2.png" alt="drawing" style="width:750px;"/>

### And it's not always enough...

<img src="./images/MMF/3.png" alt="drawing" style="width:850px;"/>

## + hours of postprocessing<a class="tocSkip">

## Physics guided numerical compensation of optical defects

### The problem

<img src="./images/MMF/4.png" alt="drawing" style="width:750px;"/>

#### M. Matthès, Y. Bromberg, J. de Rosny and S.M. Popoff, *Learning and avoiding disorder in multimode fibers*, arxiv:2010.14813 (2020), Phys. Rev. X (2021)<a class="tocSkip">

### The solution

#### Instead of spending hours/days to optimize physically the setup (slow)<a class="tocSkip"> 
    
#### Using deep learning framework to mimic Physics and find the correction numerically (fast)<a class="tocSkip">


<img src="./images/MMF/5.png" alt="drawing" style="width:950px;"/>

### Idea: Network that applies transformations that mimic aberrations/misalignments<a class="tocSkip">
    
#### Trainable parameters are the ones that characterize these effects<a class="tocSkip">

<img src="./images/MMF/6.png" alt="drawing" style="width:850px;"/>

### From Beast to Beauty

<img src="./images/MMF/7.png" alt="drawing" style="width:550px;"/>

### - Optimization: ~1 minute on a laptop, few seconds with GPUs<a class="tocSkip">
### - Alignment: ~few minutes<a class="tocSkip">

## Take home message
### If you know enought about the model of a disturbing effect, it is can be much faster and even more efficient to compensate for it numerically. <a class="tocSkip">    
### And it is nowadays quite easy with Python...<a class="tocSkip">

# Other useful stuffs

* **Loggers** (standard loggers, graphical ones; e.g. Neptune.ai)
* ***Just in time compiling*** (speed up codes with Numba!!!)
* **Git**
* **Conda** environments and package manager

## Remember: The more you share, the more there is a chance what you did will be useful<a class="tocSkip">

### More information: [www.wavefrontshaping.net](https://wavefrontshaping.net)<a class="tocSkip">
### https://github.com/wavefrontshaping/<a class="tocSkip">